In [102]:
# !pip install sqlite3

In [1]:
import pandas as pd
from tqdm import tqdm

# Создаем базу данных
добавить описание!

In [60]:
import sqlite3

def create_database(db_file):
    """Создает базу данных SQLite с заданной структурой."""

    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()

    # Создаем таблицы
    cursor.execute("""
        CREATE TABLE DOCS (
            doc_id INTEGER,
            doc_name TEXT,
            doc_text TEXT,
            gender TEXT
        )
    """)

    cursor.execute("""
        CREATE TABLE WORD (
            word_id INTEGER,
            word_position INTEGER,
            word TEXT,
            lemma TEXT,
            lemma_id INTEGER,
            tag TEXT,
            sent_id INTEGER
        )
    """)

    cursor.execute("""
        CREATE TABLE SENT (
            doc INTEGER,
            sent_id INTEGER,
            sent_position INTEGER,
            sentence_text TEXT
        )
    """)

    cursor.execute("""
        CREATE TABLE INVERTED_LEMMA_INDEX (
            sent INTEGER,
            lemma_id INTEGER
        )
    """)

    # Создаем индексы
    cursor.execute("CREATE INDEX idx_sent_id ON SENT (sent_id)")
    cursor.execute("CREATE INDEX idx_sent_doc ON SENT (doc, sent_id)") 
    cursor.execute("CREATE INDEX idx_word_id ON WORD (word_id)")
    cursor.execute("CREATE INDEX idx_lemma_id ON WORD (lemma_id)")
    cursor.execute("CREATE INDEX idx_sent_id_word ON WORD (sent_id)")
    cursor.execute("CREATE INDEX idx_inverted_lemma_index ON INVERTED_LEMMA_INDEX (sent, lemma_id)")
    cursor.execute("CREATE INDEX idx_inverted_lemma_index_lemma_id ON INVERTED_LEMMA_INDEX (lemma_id)")

    # Сохраняем изменения и закрываем подключение
    conn.commit()
    conn.close()

if __name__ == "__main__":
    db_file = "sem_notes_corp.db"  # Имя файла базы данных
    create_database(db_file)
    print(f"База данных '{db_file}' создана успешно!")

База данных 'sem_notes_corp.db' создана успешно!


# Обработка данных из большого датафрейма

Общий датафрейм содержит всю информацию, которая нужна, но его элементарная единица -- это словоформа. Поэтому придется убрать дублирование предложений и документов. 

In [4]:
df = pd.read_csv('nlp_project_corpus_df - nlp_project_corpus_df (4).csv')
df

,gender,doc_id,doc_name,doc_text,sent_id,sentence_text,sent_position,word_id,word,lemma,tag,word_position,inverted_word_index,inverted_lemma_index
0,f,1,История семантики: основные этапы и имена.,f 1. История семантики: основные этапы и имена...,1,"Из-за того, что смыслы не имеют материального ...",1,1,из-за,из-за,PR,1,"[1, 54, 171, 991]","[1, 54, 171, 991]"
1,f,1,История семантики: основные этапы и имена.,f 1. История семантики: основные этапы и имена...,1,"Из-за того, что смыслы не имеют материального ...",1,2,того,то,SPRO,2,"[1, 7, 12, 135, 147, 153, 235, 365, 399, 412, ...","[1, 2, 6, 7, 11, 12, 19, 21, 29, 43, 44, 52, 5..."
2,f,1,История семантики: основные этапы и имена.,f 1. История семантики: основные этапы и имена...,1,"Из-за того, что смыслы не имеют материального ...",1,3,что,что,CONJ,3,"[1, 19, 58, 71, 135, 141, 147, 235, 277, 292, ...","[1, 19, 58, 71, 135, 141, 147, 235, 277, 292, ..."
3,f,1,История семантики: основные этапы и имена.,f 1. История семантики: основные этапы и имена...,1,"Из-за того, что смыслы не имеют материального ...",1,4,смыслы,смысл,S,4,"[1, 265, 267, 746]","[1, 17, 38, 42, 48, 104, 265, 267, 394, 395, 4..."
4,f,1,История семантики: основные этапы и имена.,f 1. История семантики: основные этапы и имена...,1,"Из-за того, что смыслы не имеют материального ...",1,5,не,не,PART,5,"[1, 2, 7, 8, 14, 17, 18, 19, 32, 36, 43, 47, 5...","[1, 2, 7, 8, 14, 17, 18, 19, 32, 36, 43, 47, 5..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10017,f,34,Грамматика конструкций.,f 34. Грамматика конструкций. Авторы и их посл...,1144,"Конструкция – языковое выражение, у которого е...",30,10018,значения,значение,S,16,"[7, 21, 79, 98, 135, 222, 426, 432, 443, 476, ...","[7, 14, 21, 79, 98, 105, 108, 131, 135, 142, 2..."
10018,f,34,Грамматика конструкций.,f 34. Грамматика конструкций. Авторы и их посл...,1144,"Конструкция – языковое выражение, у которого е...",30,10019,или,или,CONJ,17,"[17, 18, 58, 105, 119, 120, 129, 142, 167, 170...","[17, 18, 58, 105, 119, 120, 129, 167, 170, 230..."
10019,f,34,Грамматика конструкций.,f 34. Грамматика конструкций. Авторы и их посл...,1144,"Конструкция – языковое выражение, у которого е...",30,10020,формы,форма,S,18,[1144],"[9, 38, 41, 163, 167, 177, 213, 301, 303, 488,..."
10020,f,34,Грамматика конструкций.,f 34. Грамматика конструкций. Авторы и их посл...,1144,"Конструкция – языковое выражение, у которого е...",30,10021,составных,составной,A,19,[1144],[1144]


### Датафрейм уникальных документов

In [32]:
name = []

df_docs = df.copy(deep=True)

for i in tqdm(range(len(df_docs))):
        #doc_id = i
        gender = df_docs.loc[i, 'gender']
        doc_name = df_docs.loc[i, 'doc_name']
        doc_text = df_docs.loc[i, 'doc_text']
        if doc_name not in name:
                name.append(doc_name)
        else:
                df_docs.drop(i, inplace=True)

df_docs.reset_index(inplace=True)

100%|██████████| 10022/10022 [00:21<00:00, 456.55it/s]


In [33]:
docs_ids = {}
# словарь документов {название : айди}

for i in tqdm(range(len(df_docs))):
    docs_ids[df_docs.loc[i, 'doc_name']]=i

docs_ids

100%|██████████| 31/31 [00:00<?, ?it/s]


{'История семантики: основные этапы и имена.': 0,
 'Треугольник Фреге.': 1,
 'Теория референции.': 2,
 'Понятие интенсионала и экстенсионала.': 3,
 'Компонентный анализ.': 4,
 'Фон и фигура.': 5,
 'Траектор и ориентир.': 6,
 'Семантические роли.': 7,
 'Фреймовая семантика.': 8,
 'Синкретическое выражение семантических ролей.': 9,
 'Актанты и сирконстанты.': 10,
 'Семантические и синтаксические актанты.': 11,
 'Наследование актантов.': 12,
 'Компоненты значения.': 13,
 'Понятие сферы действия.': 14,
 'Фигура наблюдателя.': 15,
 'Понятие коннотации.': 16,
 'Отрицательная и положительная поляризация.': 17,
 'Полисемия.': 18,
 'Понятие семантического инварианта.': 19,
 'Основные типы семантических сдвигов.': 20,
 'Р.Якобсон о метафоре и метонимии.': 21,
 'А.Вежбицка о метафоре и сравнении.': 22,
 'Дж.Лакофф о концептуальной метафоре.': 23,
 'Концептуальный блендинг.': 24,
 'Е.В.Падучева о механизмах метафоры и метонимии.': 25,
 'Регулярная метонимия (типы с примерами).': 26,
 'Универсальна

### Датафрейм уникальных предложений

In [34]:
df_sent = df.copy(deep=True)
len(df_sent)

10022

In [35]:
text = []

df_sent = pd.DataFrame()

c = 1

for i in tqdm(range(len(df))):
        #doc_id = i
        sent_text = df.loc[i, 'sentence_text']
        sent_position = df.loc[i, 'sent_position']
        sent_id = df.loc[i, 'sent_id']
        #print(df_sent.loc[i, 'doc_name'])
        doc = int(docs_ids[(df.loc[i, 'doc_name'])])

        if (sent_id, sent_text) not in text:
                text.append((sent_id, sent_text))
                df_sent.loc[c, 'doc'] = doc
                df_sent.loc[c, 'id'] = sent_id
                df_sent.loc[c, 'sentence_text'] = sent_text
                df_sent.loc[c, 'sent_position'] = sent_position
                c+=1
        else:
                pass

df_sent.reset_index(inplace=True)

100%|██████████| 10022/10022 [00:02<00:00, 4167.53it/s]


In [272]:
df_sent

,index,doc,id,sentence_text,sent_position
0,1,0.0,1.0,"Из-за того, что смыслы не имеют материального ...",1.0
1,2,0.0,2.0,Поэтому семантика долгое время не была настоящ...,2.0
2,3,0.0,3.0,Перелом случился в XX веке.,3.0
3,4,0.0,4.0,до XX века:,4.0
4,5,0.0,5.0,- описывали грамматику и лексику (иностранные ...,5.0
...,...,...,...,...,...
1040,1041,30.0,1140.0,Семантические фраземы:,26.0
1041,1042,30.0,1141.0,1) Идиомы (≈ сращения + единства): значения ча...,27.0
1042,1043,30.0,1142.0,"2) Коллокации (≈ сочетания, = лексические функ...",28.0
1043,1044,30.0,1143.0,3) Квазиидиомы (Ø): значения частей содержатся...,29.0


### Леммы

In [36]:
lemmas = []

df_lemmas = df.copy(deep=True)

for i in tqdm(range(len(df_lemmas))):
        #doc_id = i
        lemma = df_lemmas.loc[i, 'lemma']
        #print(df_sent.loc[i, 'doc_name'])
        inv_index = (df_lemmas['inverted_lemma_index'])
        #df_sent.loc[i, 'doc'] = doc
        if lemma not in lemmas:
                lemmas.append(lemma)
        else:
                df_lemmas.drop(i, inplace=True)

df_lemmas.reset_index(inplace=True)

lemmas_ids = {}
# словарь название : айди

for i in range(len(df_lemmas)):
    lemmas_ids[df_lemmas.loc[i, 'lemma']]=(i, df_lemmas.loc[i, 'inverted_lemma_index'])

100%|██████████| 10022/10022 [00:19<00:00, 508.06it/s]


В этом словаре хранятся данные в формате
`{'lemma': (id, [inverted indexes])}`

### Запишем инвертированные индексы лемм

In [37]:
df_inv_lemma = pd.DataFrame()
i = 1
for k, v in tqdm(list(lemmas_ids.items())):
    #print(k)
    #print(v[1].replace('[', '').replace(']', '').split(', '):)
    for ind in v[1].replace('[', '').replace(']', '').split(', '):
        #print(ind)
        #print(k)
        #df_inv_lemma['lemma_id'] = v[0]
        lemma_id = int(lemmas_ids[k][0]+1)
        df_inv_lemma.loc[i, 'lemma_id']=lemma_id
        df_inv_lemma.loc[i, 'inverted_lemma_index']=ind
        i+=1


100%|██████████| 2547/2547 [01:39<00:00, 25.55it/s] 


### ... И словоформ

In [11]:
len(df)

10022

In [28]:
invwforms = pd.DataFrame()

c = 0
for i in tqdm(range(len(df))): #len(df)
        #doc_id = i
        word = df.loc[i, 'word']
        pos = df.loc[i, 'word_position']
        w_id = df.loc[i, 'word_id']
        word_inv_i = df.loc[i, 'inverted_word_index'].replace('[', '').replace(']', '').split(', ')
        for index in word_inv_i:
                invwforms.loc[c, 'word_id'] = w_id
                invwforms.loc[c, 'sent_id'] = index
                c+=1
        #print(word)
        #if word not in wforms.keys(): 
        # wforms[word]=(df.loc[i, 'word_id'], df.loc[i, 'inverted_word_index'])
        # c+=1
        
        #print(df_sent.loc[i, 'doc_name'])
        #inv_index = (df_lemmas['inverted_lemma_index'])
        #df_sent.loc[i, 'doc'] = doc


100%|██████████| 10022/10022 [19:17:25<00:00,  6.93s/it]  


In [29]:
invwforms.to_csv('ttest.csv')

In [30]:
invwforms

,word_id,sent_id
0,1.0,1
1,1.0,54
2,1.0,171
3,1.0,991
4,2.0,1
...,...,...
1056195,10022.0,1118
1056196,10022.0,1123
1056197,10022.0,1141
1056198,10022.0,1143


In [277]:
df_inv_word = pd.DataFrame()
i = 1
for k, v in tqdm(list(wforms.items())):
    #print(k)
    #print(v[1].replace('[', '').replace(']', '').split(', '):)
    for ind in v[1].replace('[', '').replace(']', '').split(', '):
        #print(ind)
        #print(k)
        #df_inv_lemma['lemma_id'] = v[0]
        lemma_id = int(wforms[k][0])
        df_inv_word.loc[i, 'word_id']=lemma_id
        df_inv_word.loc[i, 'inverted_word_index']=ind
        i+=1


100%|██████████| 3927/3927 [01:58<00:00, 33.14it/s] 


# Загружаем данные в базу

### Загрузим данные в таблицу WORD

In [61]:
conn = sqlite3.connect('sem_notes_corp.db')
cursor = conn.cursor()

conn.execute("BEGIN TRANSACTION")

try:
        for i in range(len(df)): # загружаем слова
            word = df.loc[i, 'word']
            word_id = int(df.loc[i, 'word_id'])
            lemma = df.loc[i, 'lemma']
            lemma_id = lemmas_ids[lemma][0]+1
            tag = df.loc[i, 'tag']
            word_position = int(df.loc[i, 'word_position'])
            sent_id = int(df.loc[i, 'sent_id'])

            cursor.execute(
                """
                INSERT INTO WORD (word_id, word_position, word, lemma, lemma_id, tag, sent_id) 
                VALUES (?, ?, ?, ?, ?, ?, ?)
                """, 
                (word_id, word_position, word, lemma, lemma_id, tag, sent_id)
            )

        # Сохраняем изменения
        conn.commit()

        
except Exception as e:
        # Если произошла ошибка, откатываем изменения
        conn.rollback()
        raise e

finally:
        # Закрываем соединение
        conn.close()



### Загрузим данные в таблицу DOCS

In [62]:
conn = sqlite3.connect('sem_notes_corp.db')
cursor = conn.cursor()

conn.execute("BEGIN TRANSACTION")

try:

        for i in range(len(df_docs)): # загружаем документы
            doc_id = i+1
            gender = df_docs.loc[i, 'gender']
            doc_name = df_docs.loc[i, 'doc_name']
            doc_text = df_docs.loc[i, 'doc_text']

            cursor.execute(
                """
                INSERT INTO DOCS (doc_id, doc_name, doc_text, gender) 
                VALUES (?, ?, ?, ?)
                """, 
                (doc_id, doc_name, doc_text, gender)
            )

              
        # Сохраняем изменения
        conn.commit()

        
except Exception as e:
        # Если произошла ошибка, откатываем изменения
        conn.rollback()
        raise e

finally:
        # Закрываем соединение
        conn.close()


### Загрузим данные в таблицу SENT

In [63]:
conn = sqlite3.connect('sem_notes_corp.db')
cursor = conn.cursor()

conn.execute("BEGIN TRANSACTION")

try:
        for i in range(len(df_sent)):
            sent_id = df_sent.loc[i, 'id']
            sent_text = df_sent.loc[i, 'sentence_text']
            sent_position = int(df_sent.loc[i, 'sent_position'])
            doc = int(df_sent.loc[i, 'doc'])+1

            cursor.execute(
                """
                INSERT INTO SENT (doc, sent_id, sent_position, sentence_text) 
                VALUES (?, ?, ?, ?)
                """, 
                (doc, sent_id, sent_position, sent_text)
            )

              
        # Сохраняем изменения
        conn.commit()

        
except Exception as e:
        # Если произошла ошибка, откатываем изменения
        conn.rollback()
        raise e

finally:
        # Закрываем соединение
        conn.close()

### Загрузим данные в таблицу INVERTED_LEMMA_INDEX

In [64]:
conn = sqlite3.connect('sem_notes_corp.db')
cursor = conn.cursor()

conn.execute("BEGIN TRANSACTION")

try:
        for i in (range(len(df_inv_lemma))): # загружаем предложения
            lemma_id = int(df_inv_lemma.loc[i+1, 'lemma_id'])
            inverted_lemma_index = int(df_inv_lemma.loc[i+1, 'inverted_lemma_index'])

            cursor.execute(
                """
                INSERT INTO INVERTED_LEMMA_INDEX (sent, lemma_id) 
                VALUES (?, ?)
                """, 
                (inverted_lemma_index, lemma_id)
            )

              
        # Сохраняем изменения
        conn.commit()

        
except Exception as e:
        # Если произошла ошибка, откатываем изменения
        conn.rollback()
        raise e

finally:
        # Закрываем соединение
        conn.close()

### Загрузим данные в таблицу INVERTED_WORD_INDEX

In [58]:
invwforms

,word_id,sent_id
0,1.0,1
1,1.0,54
2,1.0,171
3,1.0,991
4,2.0,1
...,...,...
1056195,10022.0,1118
1056196,10022.0,1123
1056197,10022.0,1141
1056198,10022.0,1143


In [65]:
# conn = sqlite3.connect('sem_notes_corp.db')
# cursor = conn.cursor()

# conn.execute("BEGIN TRANSACTION")

# try:
#         for i in tqdm((range(len(invwforms)))): # загружаем предложения
#             word_id = int(invwforms.loc[i, 'word_id'])
#             inverted_word_index = int(invwforms.loc[i, 'sent_id'])

#             cursor.execute(
#                 """
#                 INSERT INTO INVERTED_WORD_INDEX (sent, word_id) 
#                 VALUES (?, ?)
#                 """, 
#                 (inverted_word_index, word_id)
#             )

              
#         # Сохраняем изменения
#         conn.commit()

        
# except Exception as e:
#         # Если произошла ошибка, откатываем изменения
#         conn.rollback()
#         raise e

# finally:
#         # Закрываем соединение
#         conn.close()

  0%|          | 0/1056200 [00:00<?, ?it/s]

OperationalError: no such table: INVERTED_WORD_INDEX

### И задаем запрос

In [66]:
import sqlite3

def get_sentences_with_lemma(db_file, lemma, tag):
    """Выбирает предложения из таблицы SENT, 
    которые содержат слова с заданной леммой.
    """

    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()

    # Запрос с несколькими JOIN
    cursor.execute("""
        SELECT DISTINCT sentence_text 
        FROM SENT 
        JOIN INVERTED_LEMMA_INDEX ON INVERTED_LEMMA_INDEX.sent = SENT.sent_id
        JOIN WORD ON INVERTED_LEMMA_INDEX.lemma_id = WORD.lemma_id
        WHERE WORD.lemma = ? and WORD.tag = ?
    """, (lemma, tag))

    # Получение результата
    result = cursor.fetchall()

    # Закрываем подключение
    conn.close()

    return result

db_file = "sem_notes_corpora.db"  # Имя файла базы данных
query_word = 'любовь'
tag = 'S'
sentences = get_sentences_with_lemma(db_file, query_word, tag)

print(f"Предложения, содержащие слово '{query_word, tag}':")
for sentence in sentences:
        print(sentence)

Предложения, содержащие слово '('любовь', 'S')':
('Любовь – это путешествие',)
